In [1]:
import torch
import torch.nn as nn
import torch.optim
import torchvision
import torch.utils.data
import numpy as np 
import matplotlib.pyplot as plt
import Autoencoder as AE
import scipy
import scipy.interpolate
import os 

# use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Reproducibility  
torch.manual_seed(0);

C:\Users\dstre\AppData\Local\Temp\ipykernel_20612\2077273905.py:7: UserWarning: Treat the new Tool classes introduced in v1.5 as experimental for now; the API and rcParam may change in future versions.
  import matplotlib.pyplot as plt


In [2]:
### Load and Process Data ###
run_interp = False 
cyl_data = np.load('./SamsStuff/data_numpy.npy') # of shape [n_samples, (u, v, vort), n_gridpoints]
xy_coords = np.load('./SamsStuff/xy_coords.npy').T # of shape [n_gridpoints, (x, y)].T
vort_data = cyl_data[:, 2, :]

# interpolate onto structured grid
x = xy_coords[0, :]
y = xy_coords[1, :]
x_bounds = [x.min(), x.max()]
y_bounds = [y.min(), y.max()]
scaling = 20 #controls resolution of structured grid
Nx = int((x_bounds[1] - x_bounds[0]) * scaling)
Ny = int((y_bounds[1] - y_bounds[0]) * scaling)
grid_x, grid_y = np.meshgrid(np.linspace(x_bounds[0], x_bounds[1], Nx), np.linspace(y_bounds[0], y_bounds[1], Ny))

# cylinder mask 
mask = np.ones_like(grid_x)
mask[np.where(grid_x**2 + grid_y**2 < 0.5**2)] = 0

if run_interp: # takes ~3m for scaling = 20
    vort_structured = np.zeros((vort_data.shape[0], Ny, Nx))
    for i in range(vort_data.shape[0]):
        interpolated = scipy.interpolate.griddata((x,y), vort_data[i], (grid_x, grid_y), method='cubic')
        vort_structured[i] = interpolated * mask
    np.save('vort_structured{}.npy'.format(scaling), vort_structured)
else:
    vort_structured = np.load('vort_structured{}.npy'.format(scaling))

im_size = vort_structured.shape[1] * vort_structured.shape[2]

### Dataloaders ###
test_perc = 0.1
split_idx = int((1-test_perc) * vort_structured.shape[1])

batch_size = 128
vort_torch = torch.from_numpy(vort_structured).float()
vort_dset = torch.utils.data.TensorDataset(vort_torch[split_idx:, :, :])
dataloader = torch.utils.data.DataLoader(vort_dset, batch_size=batch_size, shuffle=False)

vort_dset_test = torch.utils.data.TensorDataset(vort_torch[:split_idx, :, :])
dataloader_test = torch.utils.data.DataLoader(vort_dset, batch_size=batch_size, shuffle=False)


In [3]:
optimizers = [
    'Adagrad',
    'SGD',
    'SGD_momentum',
    'Adam',
    'Adam_beta0.5',
    'RMSprop',
]

# Training parameters
lrs = [1e-2, 1e-3, 1e-4, 1e-5]
EPOCHS = 200
SAVE_DIR = './train_results_cyl/trial6/'

val_losses = {}
for opt in optimizers:
    for lr in lrs:
        #create directory for results
        out_dir = SAVE_DIR + opt + '/'
        os.makedirs(out_dir, exist_ok=True)

        print('\nTraining with optimizer: {}    lr = {}'.format(opt, lr))
        #Init network
        n_latent = int(0.01* cyl_data.shape[-1]) #use 1% of original amount of data
        n_hidden = 5 #number of hidden layers 
        NN_width = 1024*4  #width of the hidden layers
        error = nn.L1Loss()
        net = AE.AutoEncoder(im_size, n_latent, n_hidden, NN_width, taper = True, square=False).to(device)

        if opt == 'SGD':
            optimizer = torch.optim.SGD(net.parameters(), lr=lr)
        elif opt == 'SGD_momentum':
            optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9, nesterov=True)
        elif opt == 'Adam':
            optimizer = torch.optim.Adam(net.parameters(), betas=(0.9, 0.999), lr=lr)
        elif opt == 'Adam_beta0.5':
            optimizer = torch.optim.Adam(net.parameters(), betas=(0.5, 0.999), lr=lr)
        elif opt == 'RMSprop':
            optimizer = torch.optim.RMSprop(net.parameters(), lr=lr)
        elif opt == 'LBFGS':
            optimizer = torch.optim.LBFGS(net.parameters(), lr=lr)
        elif opt == 'Adagrad':
            optimizer = torch.optim.Adagrad(net.parameters(), lr=lr)

        #train
        train_out = AE.train_cyl(net, optimizer, dataloader, dataloader_test, epochs=EPOCHS)

        #save results
        out_fname = out_dir + 'train_results_{}_lr={}.png'.format(train_out['optimizer'], lr)
        AE.plot_cyl_reconst(out_fname, train_out['net'], dataloader_test, Nx, Ny, grid_x, grid_y)

        #plot loss curve 
        plt.figure(figsize = (8,6))
        plt.plot(train_out['losses'])
        plt.xlabel('Iterations')
        plt.ylabel('L1 Loss')
        plt.yscale('log')
        plt.savefig(out_dir + 'loss_curve_{}_lr={}.png'.format(train_out['optimizer'], lr))
        plt.close()

        #save losses
        np.save(out_dir + 'losses_{}_lr={}.npy'.format(train_out['optimizer'], lr), train_out['losses'])

        val_losses[opt + '_' + str(lr)] = train_out['test_error']

        #empty cuda cache
        torch.cuda.empty_cache()


Training with optimizer: Adagrad    lr = 0.01
Epoch: 0/200     Iter: 0     Loss: 0.14840935170650482
Epoch: 6/200     Iter: 100     Loss: 2.005073070526123
Epoch: 13/200     Iter: 200     Loss: 1.6002305746078491
Epoch: 20/200     Iter: 300     Loss: 1.1154978275299072
Epoch: 26/200     Iter: 400     Loss: 1.0599778890609741
Epoch: 33/200     Iter: 500     Loss: 1.4518516063690186
Epoch: 40/200     Iter: 600     Loss: 0.6196743249893188
Epoch: 46/200     Iter: 700     Loss: 0.7234646081924438
Epoch: 53/200     Iter: 800     Loss: 0.2832223176956177
Epoch: 60/200     Iter: 900     Loss: 0.5216529369354248
Epoch: 66/200     Iter: 1000     Loss: 0.5064589977264404
Epoch: 73/200     Iter: 1100     Loss: 0.3058086335659027
Epoch: 80/200     Iter: 1200     Loss: 0.3269287049770355
Epoch: 86/200     Iter: 1300     Loss: 0.4564535617828369
Epoch: 93/200     Iter: 1400     Loss: 0.24728219211101532
Epoch: 100/200     Iter: 1500     Loss: 0.5250804424285889
Epoch: 106/200     Iter: 1600     Los

In [4]:
# print sorted val losses dictionary
sorted_val_losses = {k: v for k, v in sorted(val_losses.items(), key=lambda item: item[1])}
np.save(SAVE_DIR+'val_losses.npy', sorted_val_losses) #np.load(SAVE_DIR+'val_losses.npy', allow_pickle=True).item()
sorted_val_losses


{'Adam_0.0001': 0.008726282852391402,
 'Adam_0.001': 0.011560090600202482,
 'Adagrad_0.001': 0.011854360997676849,
 'Adam_beta0.5_0.0001': 0.01714270965506633,
 'RMSprop_0.0001': 0.021597522621353466,
 'Adam_1e-05': 0.10366777380307515,
 'Adam_beta0.5_1e-05': 0.1042220413684845,
 'RMSprop_1e-05': 0.10497997353474299,
 'Adagrad_0.0001': 0.10590006858110428,
 'RMSprop_0.001': 0.13626183867454528,
 'Adagrad_1e-05': 0.13760856886704761,
 'SGD_momentum_0.01': 0.14520154595375062,
 'SGD_0.01': 0.14824874103069305,
 'SGD_momentum_0.001': 0.14825064142545064,
 'SGD_0.001': 0.14932768940925598,
 'SGD_momentum_0.0001': 0.1493283063173294,
 'SGD_0.0001': 0.14947597483793895,
 'SGD_momentum_1e-05': 0.14947610398133596,
 'SGD_1e-05': 0.14949079950650532,
 'Adagrad_0.01': 0.3035383433103561,
 'Adam_beta0.5_0.01': 0.3536389301220576,
 'Adam_beta0.5_0.001': 12.834009416898091,
 'Adam_0.01': 1547545.9333333333,
 'RMSprop_0.01': 329855612791.4667}